**Dealing with a large dataset on your computer** is a common challenge for data analysts. If you need to process or filter the data you can set the `chunksize=` argument of pandas `read_csv()` method to loop through and work with manageable chunks of the data.

If for example, you wanted to work with a large data file (HCPS_data.csv) to pull just the rows where the `HCPS Code` is **99213**, you could read that file to a chunk of 100,000 rows at a time, filter each chunk to the rows with the specified code, save the filtered results of each chunk to a list and concatenate them all together at the end. The syntax would look something like this:

```
code_99213_rows =[]

for chunk in pd.read_csv('HCPS_data.csv', chunksize = 100000):
    code_99213_rows.append(chunk[chunk['HCPS Code'] == '99213']) 
               
code_99213_df = pd.concat(code_99213_rows, ignore_index=True)
```
======================================================================   

To shrink the size of a file so that it loads more quickly, converting a text file (CSV) to binary might make sense. In python, you can work with data to minimize its footprint and then store the resulting object (dataframe) as a [pickle](https://docs.python.org/3/library/pickle.html) file.

In [1]:
import pandas as pd
import pickle

In [2]:
%%time
may = pd.read_csv('../data/may.csv')
may.head()

Wall time: 30 s


,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-05-01 00:01:41.247000,36.136822,-86.799877,PoweredLIRL1,Powered,93.0,scooter,0.0,Bird
1,2019-05-01 00:01:41.247000,36.191252,-86.772945,PoweredXWRWC,Powered,35.0,scooter,0.0,Bird
2,2019-05-01 00:01:41.247000,36.144752,-86.806293,PoweredMEJEH,Powered,90.0,scooter,0.0,Bird
3,2019-05-01 00:01:41.247000,36.162056,-86.774688,Powered1A7TC,Powered,88.0,scooter,0.0,Bird
4,2019-05-01 00:01:41.247000,36.150973,-86.783109,Powered2TYEF,Powered,98.0,scooter,0.0,Bird


### Now try to reduce the size of the file
- objects require the most space


In [3]:
may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20292503 entries, 0 to 20292502
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   sumdtype     object 
 5   chargelevel  float64
 6   sumdgroup    object 
 7   costpermin   float64
 8   companyname  object 
dtypes: float64(4), object(5)
memory usage: 1.4+ GB


#### convert the company name to an integer 
- find the unique company names
- assign each company an integer (you can use a dictionary for this step)
- update the `companyname` column to store the integer id for each company

In [4]:
may.companyname.unique()

array(['Bird', 'Lyft', 'Gotcha', 'Lime', 'Spin', 'Jump', 'Bolt'],
      dtype=object)

In [5]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}

In [6]:
may.companyname = may.companyname.replace(company_dict)

#### next convert `pubdatetime` to a datetime 

In [7]:
may.pubdatetime = pd.to_datetime(may.pubdatetime)
may.head(2)

,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,Powered,93.0,scooter,0.0,0
1,2019-05-01 00:01:41.247,36.191252,-86.772945,PoweredXWRWC,Powered,35.0,scooter,0.0,0


#### Next remove unneeded data
#### keep just the scooters

In [8]:
may.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [39]:
may['sumdgroup'].value_counts()

scooter    17542872
Scooter     2740710
bicycle        8921
Name: sumdgroup, dtype: int64

In [9]:
may_scooters = may.loc[may.sumdgroup.isin(['scooter', 'Scooter'])]

#### keep just the columns we want to work with

In [10]:
may_scooters = may_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

#### check `.info()` again

In [11]:
may_scooters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20283582 entries, 0 to 20292502
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.1+ GB


#### The only object datatype remaining is sumdid (an alphanumeric unique identifier)
- time to pickle

In [13]:
may_scooters.to_pickle("../data/may.pkl")

In [14]:
%%time
may_test = pd.read_pickle("../data/may.pkl")

Wall time: 1.73 s


#repeating the process for june and july files

In [15]:
%%time
june = pd.read_csv('../data/june.csv')
june.head()

Wall time: 44.4 s


,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-06-01 00:00:12,36.1202,-86.7534,Powered93627c35-0f62-5b81-a78d-75a4a92ecf47,Powered,90.0,scooter,0.06,Jump
1,2019-06-01 00:00:12,36.1630,-86.7765,Powered17715097-e8a0-5494-a5ab-9b625796607d,Powered,63.0,scooter,0.06,Jump
2,2019-06-01 00:00:12,36.1202,-86.7533,Powerede5cb95ae-b091-5a93-86fa-ededd946d0d7,Powered,77.0,scooter,0.06,Jump
3,2019-06-01 00:00:12,36.1201,-86.7530,Powered71fa5e4f-1e17-54c4-936d-330df38cc2fa,Powered,0.0,scooter,0.06,Jump
4,2019-06-01 00:00:12,36.1622,-86.7806,Poweredfa549dd6-40bb-5757-ac87-2c2528f2bc68,Powered,2.0,scooter,0.06,Jump


In [16]:
june.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28046095 entries, 0 to 28046094
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   sumdtype     object 
 5   chargelevel  float64
 6   sumdgroup    object 
 7   costpermin   float64
 8   companyname  object 
dtypes: float64(4), object(5)
memory usage: 1.9+ GB


In [17]:
june.companyname.unique()

array(['Jump', 'Bird', 'Bolt', 'Gotcha', 'Spin', 'Lime', 'Lyft'],
      dtype=object)

In [19]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
june.companyname = june.companyname.replace(company_dict)

In [20]:
june.pubdatetime = pd.to_datetime(june.pubdatetime)
june.head(2)

,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-06-01 00:00:12,36.1202,-86.7534,Powered93627c35-0f62-5b81-a78d-75a4a92ecf47,Powered,90.0,scooter,0.06,5
1,2019-06-01 00:00:12,36.1630,-86.7765,Powered17715097-e8a0-5494-a5ab-9b625796607d,Powered,63.0,scooter,0.06,5


In [21]:
june.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [22]:
june_scooters = june.loc[june.sumdgroup.isin(['scooter', 'Scooter'])]

In [23]:
june_scooters = june_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

In [24]:
june_scooters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28037408 entries, 0 to 28046094
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.5+ GB


In [25]:
june_scooters.to_pickle("../data/june.pkl")

In [26]:
%%time
june_test = pd.read_pickle("../data/june.pkl")

Wall time: 2.58 s


In [27]:
%%time
july = pd.read_csv('../data/july.csv')
july.head()

Wall time: 47.5 s


,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550000,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,Lyft
1,2019-07-01 00:00:34.973000,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,Lyft
2,2019-07-01 00:00:41.183000,36.179319,-86.751538,Powered570380,Powered,76.0,scooter,0.15,Lyft
3,2019-07-01 00:00:41.620000,36.152111,-86.803821,Powered240631,Powered,43.0,scooter,0.15,Lyft
4,2019-07-01 00:00:45.087000,36.149355,-86.797550,Powered970404,Powered,52.0,scooter,0.15,Lyft


In [28]:
july.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25075445 entries, 0 to 25075444
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   sumdtype     object 
 5   chargelevel  float64
 6   sumdgroup    object 
 7   costpermin   float64
 8   companyname  object 
dtypes: float64(4), object(5)
memory usage: 1.7+ GB


In [29]:
july.companyname.unique()

array(['Lyft', 'Bird', 'Spin', 'Bolt', 'Jump', 'Lime', 'Gotcha'],
      dtype=object)

In [30]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
july.companyname = july.companyname.replace(company_dict)

In [31]:
july.pubdatetime = pd.to_datetime(july.pubdatetime)
july.head(2)

,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,1
1,2019-07-01 00:00:34.973,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,1


In [32]:
july.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [33]:
july_scooters = july.loc[july.sumdgroup.isin(['scooter', 'Scooter'])]

In [34]:
july_scooters = july_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

In [35]:
july_scooters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25066524 entries, 0 to 25075444
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.3+ GB


In [36]:
july_scooters.to_pickle("../data/july.pkl")

In [37]:
%%time
july_test = pd.read_pickle("../data/july.pkl")

Wall time: 2.14 s
